In [11]:
import requests
import pandas as pd
from datetime import datetime

In [ ]:
baseurl = "http://api.weatherapi.com/v1"
Aline_Key = "your key"
# http://api.weatherapi.com/v1/current.json?key=<YOUR_API_KEY>&q={city}
city = "Louisville"

In [13]:
response = requests.get(f" http://api.weatherapi.com/v1/current.json?key={Aline_Key}&q={city}")
response.status_code

200

In [14]:
data = response.json()
data['current']

{'last_updated_epoch': 1767129300,
 'last_updated': '2025-12-30 16:15',
 'temp_c': -1.1,
 'temp_f': 30.0,
 'is_day': 1,
 'condition': {'text': 'Overcast',
  'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png',
  'code': 1009},
 'wind_mph': 9.8,
 'wind_kph': 15.8,
 'wind_degree': 275,
 'wind_dir': 'W',
 'pressure_mb': 1022.0,
 'pressure_in': 30.19,
 'precip_mm': 0.0,
 'precip_in': 0.0,
 'humidity': 43,
 'cloud': 100,
 'feelslike_c': -5.9,
 'feelslike_f': 21.3,
 'windchill_c': -5.3,
 'windchill_f': 22.4,
 'heatindex_c': -0.9,
 'heatindex_f': 30.4,
 'dewpoint_c': -9.4,
 'dewpoint_f': 15.2,
 'vis_km': 16.0,
 'vis_miles': 9.0,
 'uv': 0.4,
 'gust_mph': 11.9,
 'gust_kph': 19.1,
 'short_rad': 339.7,
 'diff_rad': 109.78,
 'dni': 0.0,
 'gti': 105.28}

In [36]:
forecast_url = f"{baseurl}/forecast.json?key={Aline_Key}&q={city}&days=3"
forecast_response = requests.get(forecast_url)
forecast_response.status_code



200

In [39]:
forecast_data = forecast_response.json()
forecast = forecast_data['forecast']['forecastday']

In [41]:
forecast_3_days_summary = []
for day in forecast:
    forecast_3_days_summary.append(f"{day['date']}: High {day['day']['maxtemp_f']}F, Low {day['day']['maxtemp_f']}F - {day['day']['condition']['text']}")

In [42]:
forecast_3_days_summary

['2025-12-30: High 31.3F, Low 31.3F - Partly Cloudy ',
 '2025-12-31: High 40.3F, Low 40.3F - Partly Cloudy ',
 '2026-01-01: High 40.1F, Low 40.1F - Cloudy ']

In [44]:
forecast_3_days_summary = []
for day in forecast:
    date_obj = datetime.strptime(day['date'], '%Y-%m-%d')
    formatted_date = date_obj.strftime('%A, %B %#d')
    forecast_3_days_summary.append(f"{formatted_date}: High {day['day']['maxtemp_f']}F/ Low {day['day']['maxtemp_f']}F - {day['day']['condition']['text']}")

forecast_3_days_summary

['Tuesday, December 30: High 31.3F/ Low 31.3F - Partly Cloudy ',
 'Wednesday, December 31: High 40.3F/ Low 40.3F - Partly Cloudy ',
 'Thursday, January 1: High 40.1F/ Low 40.1F - Cloudy ']

In [ ]:
today = pd.Timestamp.now().strftime('%Y-%m-%d')
astronomy_url = f"{baseurl}/astronomy.json?key={Aline_Key}&q={city}&dt={today}"

In [52]:
def get_city_data(city, Aline_Key):
   
    # current weather
    current_url = f"{baseurl}/current.json?key={Aline_Key}&q={city}"
    current_data = requests.get(current_url).json()

    # forecast
    forecast_url = f"{baseurl}/forecast.json?key={Aline_Key}&q={city}&days=3"
    forecast_data = requests.get(forecast_url).json()
    forecast_days = forecast_data['forecast']['forecastday']

    forecast_summary = []
    for day in forecast_days:
        forecast_summary.append(
            f"{day['date']}: "
            f"High {day['day']['maxtemp_f']}F / "
            f"Low {day['day']['mintemp_f']}F - "
            f"{day['day']['condition']['text']}"
        )

    # astronomy
    today = pd.Timestamp.now().strftime("%Y-%m-%d")
    astronomy_url = f"{baseurl}/astronomy.json?key={Aline_Key}&q={city}&dt={today}"
    astronomy_data = requests.get(astronomy_url).json()

    # final summary dictionary
    city_summary = {
        "city": city,
        "current_temp_f": current_data['current']['temp_f'],
        "condition": current_data['current']['condition']['text'],
        "forecast_3_days": forecast_summary,
        "sunrise": astronomy_data['astronomy']['astro']['sunrise'],
        "sunset": astronomy_data['astronomy']['astro']['sunset']
    }

    return city_summary


In [54]:
city_data = get_city_data("Louisville", Aline_Key)



In [55]:
Summary_city_df = pd.DataFrame([city_data])


In [57]:
Summary_city_df .head()

,city,current_temp_f,condition,forecast_3_days,sunrise,sunset
0,Louisville,28.9,Partly cloudy,[2025-12-30: High 31.3F / Low 18.5F - Partly C...,08:00 AM,05:32 PM


In [58]:
Summary_city_df.to_csv("summary_data.csv", index=False)
